In [1]:
!pip install py3langid

In [ ]:
import pandas as pd
import numpy as np
import nltk
import py3langid as langid
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.probability import FreqDist
import pickle

In [14]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("names")
nltk.download("wordnet")
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('tagsets')
nltk.download('tagsets_json')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package names to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_

True

In [15]:
stopwords = nltk.corpus.stopwords.words("english")
names = nltk.corpus.names.words()
names.remove("Win") # Esto se agrega para evitar que se confunda con el stem/lemma "win"

In [16]:
df = pd.read_csv("spam_ham_dataset.csv")
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [17]:
df["Category"].value_counts()/len(df["Category"])

Category
ham     0.865937
spam    0.134063
Name: count, dtype: float64

In [18]:
df["Prompt Size"] = df["Message"].apply(lambda x: len(x))
df

,Category,Message,Prompt Size
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,160
5568,ham,Will ü b going to esplanade fr home?,36
5569,ham,"Pity, * was in mood for that. So...any other s...",57
5570,ham,The guy did some bitching but I acted like i'd...,125


In [19]:
df["Language"] = df["Message"].apply(lambda x: langid.classify(x)[0])
df

,Category,Message,Prompt Size,Language
0,ham,"Go until jurong point, crazy.. Available only ...",111,en
1,ham,Ok lar... Joking wif u oni...,29,mt
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,en
3,ham,U dun say so early hor... U c already then say...,49,en
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,en
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,160,en
5568,ham,Will ü b going to esplanade fr home?,36,en
5569,ham,"Pity, * was in mood for that. So...any other s...",57,en
5570,ham,The guy did some bitching but I acted like i'd...,125,en


In [20]:
df = df[df["Language"] == "en"]
df

,Category,Message,Prompt Size,Language
0,ham,"Go until jurong point, crazy.. Available only ...",111,en
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,en
3,ham,U dun say so early hor... U c already then say...,49,en
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,en
5,spam,FreeMsg Hey there darling it's been 3 week's n...,147,en
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,160,en
5568,ham,Will ü b going to esplanade fr home?,36,en
5569,ham,"Pity, * was in mood for that. So...any other s...",57,en
5570,ham,The guy did some bitching but I acted like i'd...,125,en


In [21]:
df["Category"].value_counts()/len(df["Category"])

Category
ham     0.857225
spam    0.142775
Name: count, dtype: float64

In [22]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [23]:
tokens_unwanted = ["gt", "lt"] # Esto se agrega porque son tokens frecuentes basura

In [24]:
# Tag dictionary example
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [25]:
def get_tokens(text):

  tokens = word_tokenize(text)
  cleaned_tokens = []

  for token in tokens:
    if token in stopwords: continue
    if token in names: continue
    if not token.isalpha(): continue
    if token in tokens_unwanted: continue
    token = token.lower()
    token = lemmatizer.lemmatize(token)
    cleaned_tokens.append(token)

  cleaned_tokens = [i[0] for i in pos_tag(cleaned_tokens) if i[1][:2] in ["NN", "VB", "RB", "JJ"]]

  return cleaned_tokens


In [26]:
# Ejemplo
get_tokens("Win money clicking in this link right now!!!!! Hurry!!!!")

['win', 'money', 'clicking', 'link', 'right', 'hurry']

In [27]:
df

,Category,Message,Prompt Size,Language
0,ham,"Go until jurong point, crazy.. Available only ...",111,en
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,en
3,ham,U dun say so early hor... U c already then say...,49,en
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,en
5,spam,FreeMsg Hey there darling it's been 3 week's n...,147,en
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,160,en
5568,ham,Will ü b going to esplanade fr home?,36,en
5569,ham,"Pity, * was in mood for that. So...any other s...",57,en
5570,ham,The guy did some bitching but I acted like i'd...,125,en


In [28]:
vocabulary = {category:[] for category in df["Category"].unique()}
full_vocabulary = []
for _, row in df.iterrows():
  tokens = get_tokens(row["Message"])
  vocabulary[row["Category"]] += tokens
  full_vocabulary += tokens

In [29]:
most_common_spam_tokens = [i[0] for i in FreqDist(vocabulary["spam"]).most_common(500)]
most_common_ham_tokens = [i[0] for i in FreqDist(vocabulary["ham"]).most_common(500)]

In [30]:
with open("most_common_spam.tokens", "wb") as file:
  pickle.dump(most_common_spam_tokens, file)

In [31]:
with open("most_common_ham.tokens", "wb") as file:
  pickle.dump(most_common_ham_tokens, file)

In [32]:
most_common_tokens = list(set(most_common_spam_tokens + most_common_ham_tokens))
print(len(most_common_tokens))

839


In [33]:
def get_tokens_training(text):

  tokens = word_tokenize(text)
  cleaned_tokens = []

  for token in tokens:
    if token in stopwords: continue
    if token in names: continue
    if not token.isalpha(): continue
    token = token.lower()
    token = lemmatizer.lemmatize(token)
    if token not in most_common_tokens: continue
    cleaned_tokens.append(token)

  return " ".join(cleaned_tokens)


In [34]:
# Ejemplo
get_tokens_training("Win money clicking in this link right now!!!!! Hurry!!!!")

'win money link right'

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [36]:
df = df.reset_index(drop=True)

In [37]:
df["Tokens String"] = df["Message"].apply(lambda x: get_tokens_training(x))
df

,Category,Message,Prompt Size,Language,Tokens String
0,ham,"Go until jurong point, crazy.. Available only ...",111,en,go point crazy available n great world e got wat
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,en,free entry wkly comp win fa cup final tkts tex...
2,ham,U dun say so early hor... U c already then say...,49,en,u dun say early u c already say
3,ham,"Nah I don't think he goes to usf, he lives aro...",61,en,i think go life
4,spam,FreeMsg Hey there darling it's been 3 week's n...,147,en,freemsg hey week word back i like fun still ok...
...,...,...,...,...,...
5199,spam,This is the 2nd time we have tried 2 contact u...,160,en,time tried contact u pound prize claim easy ca...
5200,ham,Will ü b going to esplanade fr home?,36,en,ü b going home
5201,ham,"Pity, * was in mood for that. So...any other s...",57,en,so
5202,ham,The guy did some bitching but I acted like i'd...,125,en,guy i like something else next week u free


In [38]:
df_training = pd.concat([
  df[df["Category"] == "ham"].sample(743),
  df[df["Category"] == "spam"].sample(743)
]).reset_index(drop=True)
df_training

,Category,Message,Prompt Size,Language,Tokens String
0,ham,Please sen :)my kind advice :-)please come her...,58,en,please kind please come try
1,ham,I'm gonna say no. Sorry. I would but as normal...,119,en,i gon na say sorry i starting time sorry are s...
2,ham,"Also north carolina and texas atm, you would j...",109,en,also go pay test result sent
3,ham,Okey dokey swashbuckling stuff what oh.,39,en,stuff oh
4,ham,Thank you princess! You are so sexy...,38,en,thank princess sexy
...,...,...,...,...,...
1481,spam,"As a valued customer, I am pleased to advise y...",152,en,valued customer i pleased advise following rec...
1482,spam,8007 25p 4 Alfie Moon's Children in Need song ...,165,en,child need song ur mob tell ur txt tone charit...
1483,spam,Todays Voda numbers ending 5226 are selected t...,156,en,today number ending selected receive award mat...
1484,spam,LORD OF THE RINGS:RETURN OF THE KING in store ...,140,en,ring store now reply win cd reply stop end txts


In [39]:
vectorizer = TfidfVectorizer(vocabulary=most_common_tokens)
X = vectorizer.fit_transform(df_training["Tokens String"])

In [40]:
with open("vectorizer.pkl", "wb") as file:
  pickle.dump(vectorizer, file)

In [41]:
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10626 stored elements and shape (1486, 839)>

In [42]:
y = df_training["Category"]

In [43]:
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10626 stored elements and shape (1486, 839)>

In [44]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [46]:
X_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 8471 stored elements and shape (1188, 839)>

In [47]:
model = GaussianNB()

In [48]:
model.fit(X_train.toarray(), y_train)

,priors,None
,var_smoothing,1e-09


In [49]:
model.predict(X_test.toarray())

array(['ham', 'ham', 'ham', 'spam', 'spam', 'ham', 'ham', 'spam', 'ham',
       'ham', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'ham',
       'spam', 'ham', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam',
       'spam', 'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham',
       'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham',
       'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'spam', 'spam', 'spam',
       'ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'spam', 'ham', 'ham',
       'ham', 'ham', 'spam', 'spam', 'spam', 'spam', 'ham', 'ham', 'spam',
       'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam',
       'ham', 'spam', 'spam', 'spam', 'ham', 'spam', 'ham', 'spam',
       'spam', 'ham', 'spam', 'spam', 'ham', 'ham', 'spam', 'spam', 'ham',
       'ham', 'ham', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'ham',
       'ham', 'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'ham',
       'ham', 'spam', 'spam', 'spam', 'spam', 'ham', 'ham', 's

In [50]:
accuracy_score(y_test, model.predict(X_test.toarray()))

0.9261744966442953

In [51]:
precision_score(y_test, model.predict(X_test.toarray()), pos_label="spam")

0.9205298013245033

In [52]:
recall_score(y_test, model.predict(X_test.toarray()), pos_label="spam")

0.9328859060402684

In [53]:
confusion_matrix(y_test, model.predict(X_test.toarray()))

array([[137,  12],
       [ 10, 139]])

In [54]:
print(classification_report(y_test, model.predict(X_test.toarray())))

              precision    recall  f1-score   support

         ham       0.93      0.92      0.93       149
        spam       0.92      0.93      0.93       149

    accuracy                           0.93       298
   macro avg       0.93      0.93      0.93       298
weighted avg       0.93      0.93      0.93       298



In [55]:
with open("model.pkl", "wb") as file:
  pickle.dump(model, file)

##"Modelo en producción"

In [56]:
import pandas as pd
import nltk
import py3langid as langid
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.probability import FreqDist
import pickle
from sklearn.naive_bayes import GaussianNB

In [57]:
with open("model.pkl", "rb") as file:
  loaded_model = pickle.load(file)
with open("vectorizer.pkl", "rb") as file:
  loaded_vectorizer = pickle.load(file)
with open("most_common_spam.tokens", "rb") as file:
  loaded_most_common_spam_tokens = pickle.load(file)
with open("most_common_ham.tokens", "rb") as file:
  loaded_most_common_ham_tokens = pickle.load(file)

In [58]:
nltk.download("stopwords")
nltk.download("names")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package names to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [59]:
lemmatizer = WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words("english")
names = nltk.corpus.names.words()

In [60]:
most_common_tokens = list(set(loaded_most_common_spam_tokens + loaded_most_common_ham_tokens))

In [61]:
def get_tokens_predict(text):

  tokens = word_tokenize(text)
  cleaned_tokens = []

  for token in tokens:
    if token in stopwords: continue
    if token in names: continue
    if not token.isalpha(): continue
    token = token.lower()
    token = lemmatizer.lemmatize(token)
    if token not in most_common_tokens: continue
    cleaned_tokens.append(token)

  return " ".join(cleaned_tokens)


In [62]:
predict_df = pd.read_csv("spam_ham_dataset.csv")

In [63]:
predict_df["Tokens String"] = predict_df["Message"].apply(lambda x: get_tokens_predict(x))

In [64]:
X = loaded_vectorizer.transform(predict_df["Tokens String"])

In [65]:
predictions = loaded_model.predict(X.toarray())

In [66]:
pd.concat([predict_df, pd.Series(predictions, name="Prediction")], axis=1)

,Category,Message,Tokens String,Prediction
0,ham,"Go until jurong point, crazy.. Available only ...",go point crazy available n great world e got wat,ham
1,ham,Ok lar... Joking wif u oni...,ok lar wif u,ham
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts tex...,spam
3,ham,U dun say so early hor... U c already then say...,u dun say early u c already say,ham
4,ham,"Nah I don't think he goes to usf, he lives aro...",i think go life,ham
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,time tried contact u pound prize claim easy ca...,spam
5568,ham,Will ü b going to esplanade fr home?,ü b going home,ham
5569,ham,"Pity, * was in mood for that. So...any other s...",so,ham
5570,ham,The guy did some bitching but I acted like i'd...,guy i like something else next week u free,ham
